m# RETO RAG — Implementación End-to-End (Paso 3)

Este notebook implementa un **sistema RAG (Retrieval-Augmented Generation)** en entorno local (Anaconda + PyCharm).
Incluye: ingestión de PDF, chunking, embeddings, base vectorial **ChromaDB**, y pipeline de Recuperación + Generación.

**Requisitos mínimos**: `pip install --no-cache-dir langchain langchain-openai chromadb pypdf`
(Opcionales: `tiktoken`, `papermill`)

## 0) Configuración inicial

In [ ]:
import os
from dotenv import load_dotenv
#cargar variables desde .env
load_dotenv()
print("Api Key presente:", "OPENAI_API_KEY" in os.environ,)


In [ ]:

import os
from pathlib import Path

PDFS_DIR = Path("docs")
PDF_FILE = PDFS_DIR / "ejemplo.pdf"
CHROMA_PERSIST_DIR = Path("./.chroma_db_local")

OPENAI_EMBED_MODEL = "text-embedding-3-small"
OPENAI_CHAT_MODEL  = "gpt-4o-mini"

PDFS_DIR.mkdir(exist_ok=True, parents=True)
CHROMA_PERSIST_DIR.mkdir(exist_ok=True, parents=True)

print("OPENAI_API_KEY presente:", bool(os.getenv("OPENAI_API_KEY")))


## 1) Ingesta de PDF → Texto

In [ ]:

from pypdf import PdfReader

def extract_text_from_pdf(pdf_path: Path) -> str:
    reader = PdfReader(str(pdf_path))
    return "\n".join([p.extract_text() or "" for p in reader.pages])

if PDF_FILE.exists():
    raw_text = extract_text_from_pdf(PDF_FILE)
    print("Longitud del texto extraído:", len(raw_text))
else:
    raw_text = ""
    print(f"Coloca un PDF en {PDF_FILE} y vuelve a ejecutar esta celda.")


## 2) Chunking

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text: str, chunk_size=500, chunk_overlap=80):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
        length_function=len,
    )
    return splitter.split_text(text)

chunks = chunk_text(raw_text) if raw_text else []
print(f"Total de fragmentos: {len(chunks)}")
if chunks[:1]:
    print("Ejemplo de fragmento:\n", chunks[0][:200], "...")


## 3) Embeddings + ChromaDB

In [ ]:

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

def build_vector_store(text_chunks, persist_dir: Path):
    if not text_chunks:
        raise ValueError("No hay fragmentos para indexar. Verifica el PDF y el chunking.")
    emb = OpenAIEmbeddings(model=OPENAI_EMBED_MODEL)
    # Al crear con persist_directory, Chroma persiste automáticamente (>=0.4)
    db = Chroma.from_texts(text_chunks, emb, persist_directory=str(persist_dir))
    return db

db = None
if chunks:
    db = build_vector_store(chunks, CHROMA_PERSIST_DIR)
    print("ChromaDB inicializada en:", CHROMA_PERSIST_DIR)
else:
    print("No se construyó la base vectorial (sin fragmentos).")


## 4) Recuperación + Generación

In [ ]:

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

def build_rag_chain(chroma_db, chat_model=OPENAI_CHAT_MODEL, k=4):
    llm = ChatOpenAI(model=chat_model)
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    qa = RetrievalQA.from_chain_type(
        llm=llm, retriever=retriever, return_source_documents=True
    )
    return qa

qa = build_rag_chain(db) if db else None
if qa: print("Pipeline RAG listo.")


## 5) Prueba rápida

In [ ]:

query = "¿Cuáles son los conceptos clave mencionados en el documento?"
if qa and os.getenv("OPENAI_API_KEY"):
    result = qa.invoke({"query": query})
    print("Respuesta:", result["result"])
    print("\n---\nFragmentos usados:")
    for i, doc in enumerate(result["source_documents"], start=1):
        print(f"[{i}] {doc.page_content[:200]}...")
elif qa and not os.getenv("OPENAI_API_KEY"):
    print("Configura OPENAI_API_KEY antes de consultar.")


In [ ]:
query = "¿De que trata el concepto hsbt?"
if qa and os.getenv("OPENAI_API_KEY"):
    result = qa.invoke({"query": query})
    print("Respuesta:", result["result"])
    print("\n---\nFragmentos usados:")
    for i, doc in enumerate(result["source_documents"], start=1):
        print(f"[{i}] {doc.page_content[:200]}...")
elif qa and not os.getenv("OPENAI_API_KEY"):
    print("Configura OPENAI_API_KEY antes de consultar.")


In [ ]:
query = "¿Regla para el cálculo de Hsbt?"
if qa and os.getenv("OPENAI_API_KEY"):
    result = qa.invoke({"query": query})
    print("Respuesta:", result["result"])
    print("\n---\nFragmentos usados:")
    for i, doc in enumerate(result["source_documents"], start=1):
        print(f"[{i}] {doc.page_content[:200]}...")
elif qa and not os.getenv("OPENAI_API_KEY"):
    print("Configura OPENAI_API_KEY antes de consultar.")